<a href="https://colab.research.google.com/github/cw00dw0rd/ArangoDBUniversity/blob/master/FuzzySearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![arangodb](https://github.com/joerg84/ArangoDBUniversity/blob/master/img/ArangoDB_logo.png?raw=1)

# Fuzzy Search 

<a href="https://colab.research.google.com/github/joerg84/ArangoDBUniversity/blob/master/FuzzySearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ArangoSearch provides information retrieval features, natively integrated into ArangoDB’s query language and with support for all data models. It is primarily a full-text search engine, a much more powerful alternative to the full-text index type.
Check this [ArangoSearch notebook](https://colab.research.google.com/github/joerg84/ArangoDBUniversity/blob/master/ArangoSearch.ipynb) for an introduction to ArangoSearch.

When dealing with real-world text retrieval, we often not only care about exact matches to our search phrase but need to consider for example typos or alternative spellings.
“Fuzzy search” is an umbrella term referring to a set of algorithms for such approximate matching. Usually such algorithms evaluate some similarity measure showing how close a search term is to the items in a dictionary. Then a search engine can make a decision on which results have to be shown first.

In this notebook we will apply at two different implementation of fuzzy search in [ArangoSearch](https://www.arangodb.com/why-arangodb/full-text-search-engine-arangosearch/):
* [NGram similarity](https://www.arangodb.com/docs/devel/aql/functions-arangosearch.html#ngram_match)
* [Levenshtein distance](https://www.arangodb.com/docs/devel/aql/functions-arangosearch.html#levenshtein_match
)

# Setup 

Before getting started with ArangoSearch we need to prepare our environment and create a temporary database on ArangoDB's managed Service Oasis.

In [ ]:
%%capture
!git clone https://github.com/joerg84/ArangoDBUniversity.git
!rsync -av ArangoDBUniversity/ ./ --exclude=.git
!pip3 install pyarango
!pip3 install "python-arango>=5.0"

In [ ]:
import json
import requests
import sys
import oasis
import time

from pyArango.connection import *
from arango import ArangoClient

Create the temporary database:

In [ ]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName="FuzzyArangoSearch", credentialProvider="https://de64d9dc6b66.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB")

# Connect to the temp database
# Please note that we use the python-arango driver as it has better support for ArangoSearch 
database = oasis.connect_python_arango(login)

In [ ]:
print("https://"+login["hostname"]+":"+str(login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

Feel free to use to above URL to checkout the WebUI!

##  IMDB Example Dataset

![imdb](https://github.com/joerg84/ArangoDBUniversity/blob/master/img/IMDB_graph.png?raw=1)

Last, but not least we will import the [IMBD Example Dataset](https://github.com/arangodb/example-datasets/tree/master/Graphs/IMDB) including information about various movies, actors, directors, ... as a graph. 
*Note the included arangorestore will only work on Linux or Windows systems, if you want to run this notebook on a different OS please consider using the appropriate arangorestore from the [Download area](https://www.arangodb.com/download-major/).

## Linux:

In [ ]:
! ./tools/arangorestore -c none --server.endpoint http+ssl://{login["hostname"]}:{login["port"]} --server.username {login["username"]} --server.database {login["dbName"]} --server.password {login["password"]} --default-replication-factor 3  --input-directory "data/imdb"

# Create First View

An ArangoSearch view contains references, or links, to documents stored in different collections. These links are stored as a columnar store and inverted index to make ArangoSearch queries fast, more on links in a moment. 
ArangoSearch views makes it possible to perform complex federated searches, even over a complete graph including vertex and edge collections.

In [ ]:
# Create an ArangoSearch view.
database.create_arangosearch_view(
    name='v_imdb',
    properties={'cleanupIntervalStep': 0}
)

# The ArangoSearch View needed for our examples
database.create_arangosearch_view(name="imdb_ngram_match")


Next, we will create a custom analyzer to fill this view: 

TODO Requirements
Note that, the analyzer must have the "position" and "frequency" features enabled.  

In [ ]:
 # Retrieve list of analyzers.
database.analyzers()

# Delete any analyzer with the same names as our analyzers
database.delete_analyzer('test_analyzer', ignore_missing=True)
database.delete_analyzer('match_analyzer', ignore_missing=True)

# A test analyzer
database.create_analyzer(
        name='test_analyzer',
        analyzer_type='text',
        properties={'locale': 'en.utf-8', 'case': 'lower', 'stopwords': [], 'accent': False, 'stemming': True, 'edgeNgram' : { 'min': 3, 'max': 8, 'preserveOriginal': True }},
        features=["frequency",'norm','position']
    )

# An ngram analyzer with ngram size of 2 and the features required for NGRAM_MATCH
database.create_analyzer( 
    name="match_analyzer", 
    analyzer_type="ngram",
    properties={  
        "min": 2,  
        "max": 2,  
        "preserveOriginal": False 
        }, 
    features=["position", "frequency"] 
    )

As of now this view is empty, so we need to link it to a collection (i.e., imdb_vertices).

In [ ]:
# Populate a link object that uses the previously created match_analyzer on the title field
 link = {
    "includeAllFields": True,
    "fields": { 
        "title": { 
            "analyzers": [login["dbName"]+"::match_analyzer"]
            },
        "description": { 
            "analyzers": [login["dbName"]+"::match_analyzer"]
            }
          }
        }
# Update the imdb_ngram_match view with the link object
database.update_arangosearch_view(
    name="imdb_ngram_match",
    properties={"links": { "imdb_vertices": link }}
)

# Popluate a test link object that uses test_analyzer
 link = { 
  "includeAllFields": True,
  "fields" : { "description" : { "analyzers" : [ "test_analyzer" ] } }
}

# Update v_imdb view with the test link object
database.update_arangosearch_view(
    name='v_imdb',
    properties={'links': { 'imdb_vertices': link }}
)

As the indexing might take a few seconds, let us have a brief look at what is actually going on.

When you link a collection you can choose which individual fields to link or specify to link all fields. It might be helpful to think about linking fields in the same way you think about indexing attributes, although not exactly the same. When you link data to a view it is indexed in a way that allows for quick retrieval. This process also stores the data in a way that allows for the ArangoSearch-specific AQL functions to perform unique queries such as tokenizing, stemming, removing stop words, and as we will see in this notebook complex matching functions.

An additional benefit and a difference to typical indexing is that you are able to link multiple collections to one view and apply the desired analyzers. The image below shows how the collections are linked, analyzed and then made available via the view. When performing queries you can use all the typical AQL functions against a view, the same way that you would with a collection name. Though, the real benefit comes when using ArangoSearch-specific functions and you start taking advanatage of features such as ranking.

![ArangoSearch](https://github.com/joerg84/ArangoDBUniversity/blob/master/img/ArangoSearch_Arch.jpg?raw=1)

By now our view should be ready, so let us issue the first test query and look for short Drama Movies.
  

In [ ]:
cursor = database.aql.execute(
  """
  FOR d IN v_imdb 
    SEARCH d.type == "Movie" 
    AND 
    d.genre == "Drama" 
    AND 
    d.runtime IN 10..50 
    RETURN d.title
  """
)
# Iterate through the result cursor
for doc in cursor:
  print(doc)

If we set up everything correctly there should be 18 results, containing movies such as:
  * Frühlings Erwachen - Eine Kindertragödie
  * Glastage
  * Sunday in August



Now that we have finished some of the setup, let's move on to the functions that make up Fuzzy search. 
As mentioned in the beginning of this notebook, Fuzzy search comes in the form of various [NGram similarity](https://www.arangodb.com/docs/devel/aql/functions-arangosearch.html#ngram_match) and [Levenshtein distance](https://www.arangodb.com/docs/devel/aql/functions-arangosearch.html#levenshtein_match
) AQL functions. 

## Ngram Similarity

Ngram similarity is a measure for the difference between two strings represented by counting how long the longest sequence of matching ngrams is, divided by target’s total ngram count. To better understand this concept let's start with a simple example. The below query compares the phrase `quick fox` to the similar phrase of `quick foxx` (additional `x`). These are similar phrases and as such, they should have a high ngram similarity. 

Go ahead and execute the query below:

In [ ]:
cursor = database.aql.execute(
"""
RETURN NGRAM_SIMILARITY(
"quick fox",
"quick foxx", 
2)"""
)
# Iterate through the result cursor
for doc in cursor:
  print(doc)




With a ngram size of 2, the ngram similarity between both strings is 0.888, the closer the similarity is to 1 the more similar they are. Feel free experiment with other combinations such as `NGRAM_SIMILARITY( "same string","same string", 2)` or vary the ngramSize.

Ngram functions such as this break apart the words using the supplied ngram size, 2 in our query above. This means that the function compares the two words broken up into their 2 letter ngrams:

  ```
  quick fox         --         quick foxx
  ----------------------------------------
  qu                --         qu (match)
  ui                --         ui (match)
  ic                --         ic (match)
  ck                --         ck (match)
  k                 --         k  (match)
   f                --          f (match)
  fo                --         fo (match)
  ox                --         ox (match)
  x                 --         xx (do not match)
  ```
If we use simple math here we can see that there is around an 85% match when an extra `x` is supplied. However, N-Gram Similarity and Distance is not as simple as this, but hopefully this provides a quick intro to the basic concept of ngram matching and similarity. If you would like to take a deep dive into this topic, a paper published by [Grzegorz Kondrak at the University of Alberta](https://webdocs.cs.ualberta.ca/~kondrak/papers/spire05.pdf) is a great resource.
  

While [NGRAM_SIMILARITY()](https://www.arangodb.com/docs/devel/aql/functions-string.html#ngram_similarity) only counts fully matching ngrams, [NGRAM_POSITIONAL_SIMILARITY()](https://www.arangodb.com/docs/devel/aql/functions-string.html#ngram_positional_similarity) also considers partially matching ones. Let us look at how that effects the returned scores. 
In this first example we are comparing `NGRAM_SIMILARITY` and `NGRAM_POSITIONAL_SIMILARITY` scores using the same two phrases as with our previous  example. These phrases are so similar that counting partial matches doesn't make any difference, thus we get the same scores.

In [ ]:
cursor = database.aql.execute(
"""
RETURN
{"NGRAM_SIMILARITY" : NGRAM_SIMILARITY(
"quick fox",
"quick foxx", 
3),
"NGRAM_POSITIONAL_SIMILARITY" : NGRAM_POSITIONAL_SIMILARITY(
"quick fox",
"quick foxx", 
3)}"""
)
# Iterate through the result cursor
for doc in cursor:
  print(doc)

If we start to change a few more letters in the phrases, the differences between the two functions becomes more clear. The score for `NGRAM_POSITIONAL_SIMILARITY` is nearly double that of `NGRAM_SIMILARITY`, due to the fact that it counted the partial matches. This provides us with some additional 'fuzziness' by allowing the matching requirement to be a bit more lenient.

In [ ]:
cursor = database.aql.execute(
"""
RETURN
{"NGRAM_SIMILARITY" : NGRAM_SIMILARITY(
"quick fox",
"quirky foxx", 
3),
"NGRAM_POSITIONAL_SIMILARITY" : NGRAM_POSITIONAL_SIMILARITY(
"quick fox",
"quirky foxx", 
3)}"""
)
# Iterate through the result cursor
for doc in cursor:
  print(doc)

Depending on your requirements, the decision to count partially matching ngrams adds some 'fuzziness' that may help provide some context to your searches.

[NGRAM_SIMILARITY](https://www.arangodb.com/docs/devel/aql/functions-arangosearch.html#ngram_similarity) and [NGRAM_POSITIONAL_SIMILARITY](https://www.arangodb.com/docs/devel/aql/functions-arangosearch.html#ngram_positional_similarity) are two new functions that come with ArangoDB 3.7 and can be used to improve text searches but have a drawback of not being able to utilize the indexing benefits of views. They are still very powerful string functions and can offer a lot of functionality for text queries.

<br>


## Ngram Match

However, [NGRAM_MATCH](https://www.arangodb.com/docs/devel/aql/functions-arangosearch.html#ngram_match) is able to use the indexing of ArangoSearch views and is what we will look at next.

Let us start by using the [NGRAM_MATCH](https://www.arangodb.com/docs/devel/aql/functions-arangosearch.html#ngram_match) function to find a mispelled movie title. This should return multiple Star Wars movies, let's take a moment to understand why.

In [ ]:
cursor = database.aql.execute(
"""
FOR d IN imdb_ngram_match 
  SEARCH NGRAM_MATCH(d.title, 'Str War', 0.7, 'match_analyzer')
  SORT BM25(d) DESC
  RETURN d.title"""
)
# Iterate through the result cursor
for doc in cursor:
  print(doc)

The `NGRAM_MATCH` syntax follows typical ArangoSearch function structure. You first supply the field you would like to search, the search term(s), and then the next value is the threshold amount which must be between `0.0` and `1.0`, last is the analyzer to use on the search terms. The `.7` threshold amount is the new addition and is how much ‘fuzziness’ we are still considering to be a match.

The similarity is calculated by counting how long the longest sequence of matching ngrams is, divided by the target’s total ngram count. Only fully matching ngrams are counted.

The analyzer we used was configured with a min and max of 2, which means it looks at words 2 letters at a time. This is useful for determining the longest common sequence and context. The idea behind n-gram matching is searching for similar words, but not necessarily exact matches. One of the simplest ways of calculating similarity between two words is calculating the longest common sequence (LCS) of letters. The longer the LCS is the more similar the words are. However, this approach has one big disadvantage – absence of context. For example, words <connection> and <fonetica> have a long LCS (o-n-e-t-i) but very different meanings. To add some context, ngram sequences are used.

Each word is split into a series of letter groups and these groups are then matched. If we use the same words, but calculate similarity based on 3-grams, an ngram with max and min of 3, we will get a better similarity measure: con-onn-nne-nec-ect-cti-tio-ion vs. fon-one-net-eti-tic-ica gives shorter LCS ( zero matches). To get rid of length differences we normalize the LCS length by word length. We calculate these matches to get a rating with a value between 0 (no match at all) and 1(fully matched). 

Increasing the ngram size is not always the best choice due to it also increasing the accuracy requirement of the search. Scores would be much lower for the above Star Wars search if we had chosen an ngram size of 3. We would need to decrease our threshold requirement which can have the impact of returning less relevant results.

## Levenshtein MATCH

[Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance) is a another measure for the difference between two strings represented by the  minimum number of single-character transformations required to move from one string to the other. Let us consider a concrete example:

In [ ]:
cursor = database.aql.execute(
"""
RETURN LEVENSHTEIN_DISTANCE(
"The quick brown fox jumps over the lazy dog", 
"The quick black dog jumps over the brown fox")"""
)
# Iterate through the result cursor
for doc in cursor:
  print(doc)




Here we need a minimum of 13 transformations to move from one string to the other. 
Feel free to find a minimum sequence for this transformation or experiment with other combinations such as `LEVENSHTEIN_DISTANCE("a", "b")`.

In [ ]:
# Execute the query
cursor = database.aql.execute(
  """
  FOR doc IN v_imdb
    SEARCH LEVENSHTEIN_MATCH(
      doc.description, 
      'galxy', 
      2
      )
    RETURN doc
  """
)
# Iterate through the result cursor
for doc in cursor:
  print(doc)

Note, that we do not require to specify the analyzer as the distance as the input does not need to be mmodified.

## Comparison 

When should we choose which?


# Further Links

* https://www.arangodb.com/docs/stable/arangosearch.html

* https://www.arangodb.com/arangodb-training-center/search/arangosearch/